# @property

Этот декоратор позволяет взаимодействовать с методами как с атрибутами. Когда пользоваться декоратором @property:
<ol>
    <li>
<div style="border: 3px solid #0bcaff; padding: 2px;">
Если через этот атрибут определяются другие атрибуты и при изменении первого, зависящие от него не переопределяются сразу (например, они записаны в методе <tt>__init__</tt>, который вызывается только один раз при создании экземпляра класса)</div></li>
<li>
    <div style="border: 3px solid #aaff00; padding: 2px;">
Если мы сначала написали код без геттеров и сеттеров (изменяли значение атрибута напрямую), а потом обнаружили, что нужно добавить какое-то условие или функцию, которая обрабатывает введенное значение
    </div>
        </li>
</ol>


<div style="background: #0bcaff; text-align: center; font-size: 112%;">Первый пример: есть зависимые атрибуты</div>

In [96]:
class Employee:
    
    def __init__(self, first, last):
        self.name = first
        self.surname = last
        self.email = f'{self.name}_{self.surname}@email.com'
    
    def fullname(self):
        return f'{self.name} {self.surname}'

emp1 = Employee('John', 'Smith')

In [ ]:
@property
def name():
    

In [97]:
emp1.name, emp1.surname, emp1.email, emp1.fullname()

('John', 'Smith', 'John_Smith@email.com', 'John Smith')

In [98]:
emp1.name = 'Jorge'

In [99]:
emp1.name, emp1.surname, emp1.email, emp1.fullname()

('Jorge', 'Smith', 'John_Smith@email.com', 'Jorge Smith')

Как видим, атрибут `email`, зависящий от `name` и `surname`, не обновился, потому что был определен в ```__init__```, который был вызван только один раз при создании экземпляра и больше не вызывался (в отличие от метода ```fullname()```, который вызвался вручную после изменения атрибута и взял актуальные значения атрибутов).

Если мы хотим обращаться к `email` как к атрибуту, но при этом сделать так, чтобы при обращении к нему он вызывался заново как любая функция (брал актуальные данные как метод `fullname()`), то нужно использовать написать его как метод, обернув в декоратор `@property`:

In [1]:
class Employee:
    
    def __init__(self, first, last):
        self.name = first
        self.surname = last
    
    @property
    def email(self):
        return f'{self.name}_{self.surname}@email.com'

emp2 = Employee('Melissa', 'Smith')

In [6]:
emp2.name, emp2.surname, emp2.email

('Melissa', 'Smith', 'Melissa_Smith@email.com')

In [7]:
emp2.surname = 'Evans'
emp2.email

'Melissa_Evans@email.com'

<div style="background: #aaff00; text-align: center; font-size: 112%;">Второй пример: нужно проверить удовлетворение условия перед изменением значения</div>

In [122]:
class Color:
    
    @staticmethod
    def check(val):
        if not val:
            raise Exception("Invalid Name")
        return val
    
    
    def __init__(self, rgb_value, name):
        self.rgb_value = rgb_value
        self.name = check(name)
                
c1 = Color("#ff0000", "bright red")
print(c1.__dict__)

{'rgb_value': '#ff0000', 'name': 'bright red'}


In [123]:
c2 = Color("#ff0000", "")

Exception: Invalid Name

In [124]:
c1.name = 'blue'
c1.__dict__

{'rgb_value': '#ff0000', 'name': 'blue'}

In [125]:
c1.name = ''
c1.__dict__

{'rgb_value': '#ff0000', 'name': ''}

Как видим, реализовать нужную логику без помощью отдельного метода, созданного специального для того, чтобы изменять значение атрибута, не получится. При первой инициализации мы передали атрибуту результат функции `check(name)`, но если мы захоим изменить атрибут, ничто не помешает нам просто присвоить этому имени что угодно, и вызов при первом объявлении атрибута нам никак не поможет. Можем вызывать каждый раз этот `check(name)` вручную перед его присвоением , но тогда смысл использовать атрибут вместо метода? 

_Кстати, мы не можем для такой простой функции использовать lambda или просто написать `self.name = name if name else raise Exception('Invalid Name')`, т.к. в выражениях-генераторах и лямбда-функциях запрещены команды ``return``, `pass`, `assert`, `raise`_

In [ ]:
# используем синтаксис декоратора

# Но вообще-то, @property ограничивается не только этим...

В основе синтаксиса декоратора всегда лежит какая-то функция. То есть сначала создали функцию, потом дали ей возможно обертывать не так: `f_outside(f_inside())`
А вот так:
<code>
@f_outside
f_inside()
</code>
(Нахуя, а главное зачем - см. "Что такое декораторы, как и когда их создавать")

In [8]:
class Employee:
    
    def __init__(self, first, last):
        self.name = first
        self.surname = last
    
    def _email(self): # _"приватный метод", означает, что метод нужен 
        # для внутренней работы и не предназначен для вызова напрямую
        return f'{self.name}_{self.surname}@email.com'
    
    def _fullname(self):
        return f'{self.name} {self.surname}'
    
    email = property(_email) # а вот и сам интерефейс, который 
    # использует приватные методы, определенные выше
    fullname = property(_fullname)

emp1 = Employee('John', 'Smith')

In [9]:
emp1.name, emp1.surname, emp1.email, emp1.fullname
# пользуемся интерфейсом, предоставляемым функцией property: вместо вызова
# функции обращаемся к атрибутам

('John', 'Smith', 'John_Smith@email.com', 'John Smith')

Поэтому в первом примере атрибут `email` нужно представить как функцию, подобную `fullname()`, получающую при вызове актуальные значения. Как функцию нужно предстаивть также атрибут `name`, в которой будет заключена проверка введенного значения. Но если мы хотим сохранить интерфейс (синтаксис) обращения к атрибуту, нужно использовать функцию <tt><b>property(getter, setter, deleter)</b></tt>, который может записываться в виде декораторов <tt><b>@property</b></tt>, <tt><b>@attr_name.setter</b></tt>, <tt><b>@attr_name.deleter</b></tt>.

In [133]:
emp1.name = 'Jorge'

In [134]:
emp1.name, emp1.surname, emp1.email, emp1.fullname

('Jorge', 'Smith', 'Jorge_Smith@email.com', 'Jorge Smith')

<div style="background:#0bcaff; text-align: center; font-size: 112%;"><tt>property()</tt> для первого примера</div>

Чем еще хорошо обращение к атрибуту вместо метода: с помощью встроенного метода `__dict__` мы можем возвращать список доступных атрибутов с их значениями:

In [135]:
emp1.__dict__

{'name': 'Jorge', 'surname': 'Smith'}

Тогда как результат вызова метода или переменные класса никак в `__dict__` не фиксируются:

In [138]:
class Example:
    CLS_VAR = 'bee'
    def vmethod(self, val):
        return int(val)*3
v1 = Example()
v1.vmethod(4)
v1.__dict__

{}

<div style="background: #aaff00; text-align: center; font-size: 112%;"><tt>property()</tt> для второго примера</div>

In [127]:
class Color:

    def __init__(self, rgb_value, name):
        self._rgb_value = rgb_value
        self._name = name

    def _set_name(self, name):
        if not name:
            raise Exception("Invalid Name")
        self._name = name

    def _get_name(self):
        return self._name
    
    name = property(_get_name, _set_name)

In [128]:
c = Color("#ff0000", "bright red")
c.name # заметь, что мы не обращаемся к приватному атрибуту 
#(name вместо _name), а используем интерфейс, который предоставляет 
# функция property 

'bright red'

In [129]:
c.name = "red"
c.name

'red'

In [130]:
c.name = ''
c.name

Exception: Invalid Name

Минус использования вызова функции вместо декоратора в том, что мы все еще можем получить прямой доступ к атрибуту во втором примере, изменив его напрямую:

Есть зависимые атрибуты

Пример, когда может понадобиться декоратор **@property**:

In [41]:
class Employee:
    
    def __init__(self, first, last):
        self.name = first
        self.surname = last
    
    @property
    def email(self):
        return f'{self.name}_{self.surname}@email.com'
    
    @property
    def fullname(self):
        return f'{self.name} {self.surname}'

emp1 = Employee('John', 'Smith')

In [42]:
emp1.name, emp1.surname, emp1.email, emp1.fullname

('John', 'Smith', 'John_Smith@email.com', 'John Smith')

In [43]:
emp1.name = 'Jorge'

In [44]:
emp1.name, emp1.surname, emp1.email, emp1.fullname

('Jorge', 'Smith', 'Jorge_Smith@email.com', 'Jorge Smith')

Но теперь мы не можем обращаться к атрибуту напрямую:

In [45]:
emp1.email = "someone@gmail.com"

AttributeError: can't set attribute

Для этого придется написать еще одну функцию с декоратором, которая будет предназначена для ручной установки значения атрибута:

In [1]:
class Employee:
    
    def __init__(self, first, last):
        self.name = first
        self.surname = last
        self.__bek = 'fjdkfd'
    
    @property
    def email(self):
        print("Hello")
        return f'{self.name}_{self.surname}@email.com'
    
    @email.setter
    def email(self, email):
        self.email = email      
    
    @property
    def fullname(self):
        return f'{self.name} {self.surname}'
    
    @fullname.setter
    def fullname(self, set_name):
        first, last = set_name.split(' ')
        self.name = first
        self.surname = last
        
emp1 = Employee('John', 'Smith')

In [8]:
emp1._Employee__bek

'fjdkfd'

In [4]:
emp1.name

'John'

TypeError: __init__() missing 1 required positional argument: 'last'

In [60]:
emp1.fullname

'John Smith'

In [61]:
emp1.fullname = 'Johnny Berk'

In [62]:
emp1.fullname

'Johnny Berk'

In [63]:
emp1.email

'Johnny_Berk@email.com'

In [4]:
class WorkingMachine:
    def __init__(self) -> None:
        self._state: int
        # у атрибута есть type hint, но нет значения,
        # поскольку присвоением занимается метод setter,
        # это просто подсказка для программиста, что у объекта
        # далее будет реализовано такое свойство

    @property
    def name(self) -> int:
        """This is a silly property"""
        print(f"Machine state is {self._state}'s State")
        return self._state

    @name.setter
    def name(self, state: int) -> None:
        if state.__class__ != int:
            raise ValueError("State must be integer value")
        self._state = state
        print(f"Setting {self._state}")

    @name.deleter
    def name(self):
        del self._state
        print("Now machine without state")